# OKX Historical Market Data API

This notebook demonstrates how to fetch historical options data from OKX using their market-data-history endpoint. This data can be used for fitting option pricing models.

## API Endpoint
- **URL**: `GET /api/v5/public/market-data-history`
- **Documentation**: OKX Market Data History API

## Modules Available
1. **Trade History** - Raw trade data
2. **1-minute Candlestick** - OHLCV data
3. **Funding Rate** - Perpetual swap funding rates
6. **50-level Orderbook** - Deep orderbook snapshots


In [1]:
%load_ext autoreload
%autoreload 2

import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tqdm.auto import tqdm
import time

from utils import fetch_market_data, parse_option_instrument, get_option_combos


## Example 1: Fetch Options Trade History

Earliest available trade data is 2024-10-31

In [ ]:
start_date = datetime(2025, 10, 13)
end_date = datetime(2025, 10, 14)
# end_date = start_date + timedelta(days=5)

# Fetch trade history (module=1)
trades = fetch_market_data(1, 'OPTION', 'BTC-USD', start_date, end_date, 'daily')

# Sort by time and add datetime column
trades = trades.sort_values(['created_time']).reset_index(drop=True)
trades['datetime'] = pd.to_datetime(trades['created_time'], unit='ms')

print(f"\nFetched {len(trades)} trades.")
print(trades.info())
print(trades.head())

Fetching OPTION data (module=1) for BTC-USD
Period: 2024-10-13 00:00:00 to 2024-10-14 00:00:00
Split into 1 requests


Fetching data:   0%|          | 0/1 [00:00<?, ?it/s]

Fetch #1/1: 2 files found | Total: 2 files, 0.14 MB


✓ Successfully fetched 13985 records

Fetched 13985 trades.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13985 entries, 0 to 13984
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   instrument_name  13985 non-null  object        
 1   trade_id         13985 non-null  int64         
 2   side             13985 non-null  object        
 3   price            13985 non-null  float64       
 4   size             13985 non-null  float64       
 5   created_time     13985 non-null  int64         
 6   datetime         13985 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 764.9+ KB
None
          instrument_name  trade_id  side   price   size   created_time  \
0  BTC-USD-241018-61000-P       654  sell  0.0065   50.0  1728748847908   
1  BTC-USD-241108-58000-P       408  sell  0.0225    9.0  1728748896027   
2  BTC-USD-241013-62500-P        65   buy 

In [42]:
combos = get_option_combos(trades)

print(f"{trades['instrument_name'].nunique()} unique instruments")
print(f'{len(combos)} unique combos')
print(f'{combos["expiry"].nunique()} unique expiries from {combos["expiry"].min()} to {combos["expiry"].max()}')
print(f'{combos["strike"].nunique()} unique strikes from {combos["strike"].min()} to {combos["strike"].max()}')

# Group by expiry and get min/max strike for each date
print("Strike ranges by expiry date:")
for expiry, g in combos.groupby('expiry'):
    print(f"Expiry {expiry.strftime('%Y-%m-%d')}: ${g['strike'].min()} to ${g['strike'].max()}")

523 unique instruments
366 unique combos
13 unique expiries from 2025-10-13 00:00:00 to 2026-09-25 00:00:00
77 unique strikes from 30000 to 260000
Strike ranges by expiry date:
Expiry 2025-10-13: $96000 to $123500
Expiry 2025-10-14: $96000 to $132000
Expiry 2025-10-15: $95000 to $126000
Expiry 2025-10-16: $98000 to $126000
Expiry 2025-10-17: $92000 to $140000
Expiry 2025-10-18: $100000 to $126000
Expiry 2025-10-24: $90000 to $150000
Expiry 2025-10-31: $80000 to $190000
Expiry 2025-11-28: $80000 to $170000
Expiry 2025-12-26: $50000 to $220000
Expiry 2026-03-27: $30000 to $200000
Expiry 2026-06-26: $50000 to $260000
Expiry 2026-09-25: $60000 to $240000


## Example 2: Fetch Options Candlestick Data

Candlestick data seems to go even earlier! Although even for a few days, the dataframe is huge, and many candles have no activity.

Earliest available trade data is 2024-6-1

In [ ]:
start_date = datetime(2024, 8, 17)
end_date = start_date + timedelta(days=1)

# Fetch 1-minute candlestick data
candles = fetch_market_data(2, 'OPTION', 'BTC-USD', start_date, end_date, 'daily')

print(f"Total candles: {len(candles)}")
print("\nDataFrame info:")
print(candles.info())
candles.head()


Fetching OPTION data (module=2) for BTC-USD
Period: 2024-08-17 00:00:00 to 2024-08-18 00:00:00
Split into 1 requests


Fetching data:   0%|          | 0/1 [00:00<?, ?it/s]

Fetch #1/1: 2 files found | Total: 2 files, 5.27 MB


✓ Successfully fetched 1856783 records
Total candles: 1856783

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1856783 entries, 0 to 1856782
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   instrument_name  object 
 1   open             float64
 2   high             float64
 3   low              float64
 4   close            float64
 5   vol              int64  
 6   vol_ccy          float64
 7   vol_quote        float64
 8   open_time        int64  
 9   confirm          int64  
dtypes: float64(6), int64(3), object(1)
memory usage: 141.7+ MB
None


,instrument_name,open,high,low,close,vol,vol_ccy,vol_quote,open_time,confirm
0,BTC-USD-240818-50000-P,0.0002,0.0002,0.0002,0.0002,0,0.0,0.0,1723910400000,1
1,BTC-USD-240818-50000-P,0.0002,0.0002,0.0002,0.0002,0,0.0,0.0,1723910460000,1
2,BTC-USD-240818-50000-P,0.0002,0.0002,0.0002,0.0002,0,0.0,0.0,1723910520000,1
3,BTC-USD-240818-50000-P,0.0002,0.0002,0.0002,0.0002,0,0.0,0.0,1723910580000,1
4,BTC-USD-240818-50000-P,0.0002,0.0002,0.0002,0.0002,0,0.0,0.0,1723910640000,1


Candles are huge, so we need some processing...

In [5]:
# Drop rows with zero volume
candles = candles[candles['vol'] != 0]
candles.info()

# Massively reduced memory usuage, much better!

# Sort by time and add datetime column
candles = candles.sort_values(['open_time']).reset_index(drop=True)
candles['datetime'] = pd.to_datetime(candles['open_time'], unit='ms')

print(candles.head())
print(candles.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 12686 entries, 827 to 2332192
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   instrument_name  12686 non-null  object 
 1   open             12686 non-null  float64
 2   high             12686 non-null  float64
 3   low              12686 non-null  float64
 4   close            12686 non-null  float64
 5   vol              12686 non-null  int64  
 6   vol_ccy          12686 non-null  float64
 7   vol_quote        12686 non-null  float64
 8   open_time        12686 non-null  int64  
 9   confirm          12686 non-null  int64  
dtypes: float64(6), int64(3), object(1)
memory usage: 1.1+ MB
          instrument_name    open    high     low   close  vol  vol_ccy  \
0  BTC-USD-240607-67000-P  0.0195  0.0195  0.0195  0.0195   90     0.90   
1  BTC-USD-240726-75000-C  0.0470  0.0470  0.0470  0.0470   13     0.13   
2  BTC-USD-240628-60000-P  0.0145  0.0145  0.0145  0.

## Example 3: Fetch Orderbook Data

Orderbook data is only returned for one day.

Earliest available trade data is 2024-8-17

In [12]:
start_date = datetime(2024, 8, 17)
end_date = start_date + timedelta(days=1)

orderbook = fetch_market_data('6', 'OPTION', 'BTC-USD', start_date, end_date, 'daily')

print(f"Total snapshots: {len(orderbook)}")
print("\nDataFrame info:")
print(orderbook.info())

# Print time ranges for both time columns
print("\nTime ranges:")
print(f"timeMs range:    {pd.to_datetime(orderbook['timeMs'].min(), unit='ms')} to {pd.to_datetime(orderbook['timeMs'].max(), unit='ms')}")
print(f"exchTimeMs range: {pd.to_datetime(orderbook['exchTimeMs'].min(), unit='ms')} to {pd.to_datetime(orderbook['exchTimeMs'].max(), unit='ms')}")

print(orderbook.head())
print(orderbook.tail())


Fetching OPTION data (module=6) for BTC-USD
Period: 2024-08-17 00:00:00 to 2024-08-18 00:00:00
Split into 1 requests


Fetching data:   0%|          | 0/1 [00:00<?, ?it/s]

Fetch #1/1: 759 files found | Total: 759 files, 174.02 MB


✓ Successfully fetched 14388112 records
Total snapshots: 14388112

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14388112 entries, 0 to 14388111
Data columns (total 33 columns):
 #   Column        Dtype  
---  ------        -----  
 0   timeMs        int64  
 1   exchTimeMs    int64  
 2   ask_1_px      float64
 3   ask_1_qty     float64
 4   ask_1_ordCnt  float64
 5   ask_2_px      float64
 6   ask_2_qty     float64
 7   ask_2_ordCnt  float64
 8   ask_3_px      float64
 9   ask_3_qty     float64
 10  ask_3_ordCnt  float64
 11  symbol        object 
 12  ask_4_px      float64
 13  ask_4_qty     float64
 14  ask_4_ordCnt  float64
 15  ask_5_px      float64
 16  ask_5_qty     float64
 17  ask_5_ordCnt  float64
 18  bid_1_px      float64
 19  bid_1_qty     float64
 20  bid_1_ordCnt  float64
 21  bid_2_px      float64
 22  bid_2_qty     float64
 23  bid_2_ordCnt  float64
 24  bid_3_px      float64
 25  bid_3_qty     float64
 26  bid_3_ordCnt  float64
 27  bid_4_px      

In [14]:
# Create a DataFrame with column info
col_info = pd.DataFrame({
    'Column': orderbook.columns,
    'Dtype': [orderbook[col].dtype for col in orderbook.columns],
    'Null Count': [orderbook[col].isnull().sum() for col in orderbook.columns],
    'NA Count': [orderbook[col].isna().sum() for col in orderbook.columns], 
    'Null %': [orderbook[col].isnull().sum() / len(orderbook) for col in orderbook.columns],
    'NA %': [orderbook[col].isna().sum() / len(orderbook) for col in orderbook.columns]
})

# Display the table
print("\nColumn Information:")
print(col_info.to_string())

print(orderbook['exchTimeMs'].nunique())
print(orderbook['timeMs'].nunique())
# pmean exch - time
print((orderbook['exchTimeMs'] - orderbook['timeMs']).mean())


Column Information:
          Column    Dtype  Null Count  NA Count    Null %      NA %
0         timeMs    int64           0         0  0.000000  0.000000
1     exchTimeMs    int64           0         0  0.000000  0.000000
2       ask_1_px  float64         480       480  0.000033  0.000033
3      ask_1_qty  float64         480       480  0.000033  0.000033
4   ask_1_ordCnt  float64         480       480  0.000033  0.000033
5       ask_2_px  float64       61397     61397  0.004267  0.004267
6      ask_2_qty  float64       61397     61397  0.004267  0.004267
7   ask_2_ordCnt  float64       61397     61397  0.004267  0.004267
8       ask_3_px  float64      394249    394249  0.027401  0.027401
9      ask_3_qty  float64      394249    394249  0.027401  0.027401
10  ask_3_ordCnt  float64      394249    394249  0.027401  0.027401
11        symbol   object           0         0  0.000000  0.000000
12      ask_4_px  float64      629163    629163  0.043728  0.043728
13     ask_4_qty  float64  

In [ ]:
print(f'{orderbook["symbol"].nunique()} unique symbols: {orderbook["symbol"].unique()}')

# Create a DataFrame with just the symbol column before passing to get_option_combos
symbol_df = pd.DataFrame({'symbol': orderbook["symbol"]})
combos = get_option_combos(symbol_df)

print(f"{orderbook['symbol'].nunique()} unique instruments")
print(f'{len(combos)} unique combos')
print(f'{combos["expiry"].nunique()} unique expiries from {combos["expiry"].min()} to {combos["expiry"].max()}')
print(f'{combos["strike"].nunique()} unique strikes from {combos["strike"].min()} to {combos["strike"].max()}')

# Group by expiry and get min/max strike for each date
print("Strike ranges by expiry date:")
for expiry, g in combos.groupby('expiry'):
    print(f"Expiry {expiry.strftime('%Y-%m-%d')}: ${g['strike'].min()} to ${g['strike'].max()}")

759 unique symbols: ['BTC-USD-240818-50000-P.OK' 'BTC-USD-240818-50000-C.OK'
 'BTC-USD-240818-54000-C.OK' 'BTC-USD-240818-55500-C.OK'
 'BTC-USD-240818-54000-P.OK' 'BTC-USD-240818-56500-C.OK'
 'BTC-USD-240818-52000-P.OK' 'BTC-USD-240818-53000-P.OK'
 'BTC-USD-240818-54500-C.OK' 'BTC-USD-240818-57000-C.OK'
 'BTC-USD-240818-52000-C.OK' 'BTC-USD-240818-53000-C.OK'
 'BTC-USD-240818-56000-C.OK' 'BTC-USD-240818-57500-C.OK'
 'BTC-USD-240818-54500-P.OK' 'BTC-USD-240818-55500-P.OK'
 'BTC-USD-240818-57750-C.OK' 'BTC-USD-240818-55000-C.OK'
 'BTC-USD-240818-58000-C.OK' 'BTC-USD-240818-58250-C.OK'
 'BTC-USD-240818-56500-P.OK' 'BTC-USD-240818-55000-P.OK'
 'BTC-USD-240818-56000-P.OK' 'BTC-USD-240818-58500-C.OK'
 'BTC-USD-240818-61000-P.OK' 'BTC-USD-240818-57000-P.OK'
 'BTC-USD-240818-61500-P.OK' 'BTC-USD-240818-60500-P.OK'
 'BTC-USD-240818-58750-C.OK' 'BTC-USD-240818-62000-P.OK'
 'BTC-USD-240818-64000-P.OK' 'BTC-USD-240818-66000-C.OK'
 'BTC-USD-240818-66000-P.OK' 'BTC-USD-240818-64000-C.OK'
 'BTC-USD-2

# Example 4: Futures Data

In [19]:
start_date = datetime(2024, 8, 17)
end_date = start_date + timedelta(days=1)

futures_orderbook = fetch_market_data('6', 'FUTURES', 'BTC-USD', start_date, end_date, 'daily')

print(f"Total snapshots: {len(futures_orderbook)}")
print("\nDataFrame info:")
print(futures_orderbook.info())

Fetching FUTURES data (module=6) for BTC-USD
Period: 2024-08-17 00:00:00 to 2024-08-18 00:00:00
Split into 1 requests


Fetching data:   0%|          | 0/1 [00:00<?, ?it/s]

Fetch #1/1: 6 files found | Total: 6 files, 611.82 MB


✓ Successfully fetched 11712542 records
Total snapshots: 11712542

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11712542 entries, 0 to 11712541
Data columns (total 34 columns):
 #   Column      Dtype  
---  ------      -----  
 0   exchTimeMs  int64  
 1   bid_1_cnt   int64  
 2   bid_2_cnt   int64  
 3   bid_3_cnt   int64  
 4   bid_4_cnt   int64  
 5   bid_5_cnt   int64  
 6   timeMs      int64  
 7   bid_5_sz    float64
 8   bid_4_sz    float64
 9   bid_1_sz    float64
 10  bid_3_sz    float64
 11  bid_2_sz    float64
 12  ask_4_sz    float64
 13  ask_5_cnt   int64  
 14  ask_4_cnt   int64  
 15  ask_5_sz    float64
 16  ask_1_cnt   int64  
 17  ask_1_sz    float64
 18  ask_2_sz    float64
 19  ask_3_cnt   int64  
 20  ask_2_cnt   int64  
 21  ask_3_sz    float64
 22  maxLevel    int64  
 23  bid_5_px    float64
 24  bid_2_px    float64
 25  bid_1_px    float64
 26  bid_4_px    float64
 27  bid_3_px    float64
 28  ask_5_px    float64
 29  ask_1_px    float64
 3

In [9]:
# Drop rows where symbol starts with BTC-USDC- due to higher spreads and lower volume
futures_orderbook = futures_orderbook[~futures_orderbook['symbol'].str.startswith('BTC-USDC-')]

# Group by symbol and analyze key metrics
metrics = futures_orderbook.groupby('symbol').agg({
    'timeMs': 'count',  # Number of snapshots
    'bid_1_sz': 'mean', # Average bid size at best price
    'ask_1_sz': 'mean', # Average ask size at best price
    'bid_1_px': ['mean', 'std'],  # Price stats
    'ask_1_px': ['mean', 'std']
}).round(2)

# Calculate spreads
spreads = futures_orderbook.assign(
    spread=lambda df: df['ask_1_px'] - df['bid_1_px'],
    spread_bps=lambda df: (df['ask_1_px'] - df['bid_1_px']) / df['bid_1_px'] * 10000
).groupby('symbol').agg({
    'spread': ['mean', 'std'],
    'spread_bps': ['mean', 'std']
}).round(4)

# Combine metrics
analysis = pd.concat([metrics, spreads], axis=1)

# Sort by number of snapshots to see most active markets
print("\nMarket Analysis (sorted by number of snapshots):")
print(analysis.sort_values(('timeMs', 'count'), ascending=False).to_string())

# Print unique symbols grouped by quote currency
symbols = pd.Series(futures_orderbook['symbol'].unique())
for quote in ['USD', 'USDT']:
    quote_symbols = symbols[symbols.str.contains(quote + '-')]
    print(f"\n{quote} markets ({len(quote_symbols)}):")
    print(quote_symbols.values)



[autoreload of utils.api failed: Traceback (most recent call last):
  File "/opt/anaconda3/envs/ml_env/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 325, in check
    superreload(m, reload, self.old_objects)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/ml_env/lib/python3.13/site-packages/IPython/extensions/autoreload.py", line 580, in superreload
    module = reload(module)
  File "/opt/anaconda3/envs/ml_env/lib/python3.13/importlib/__init__.py", line 129, in reload
    _bootstrap._exec(spec, module)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 1022, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1160, in get_code
  File "<frozen importlib._bootstrap_external>", line 1090, in source_to_code
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/Users/pascalandreas/Documents/reposi


Market Analysis (sorted by number of snapshots):
                     timeMs bid_1_sz ask_1_sz  bid_1_px          ask_1_px           spread          spread_bps        
                      count     mean     mean      mean     std      mean     std     mean      std       mean     std
symbol                                                                                                                
BTC-USD-240927.OK   4011497    20.18    17.79  59702.07  150.85  59709.43  151.26   7.3613   5.3567     1.2329  0.8969
BTC-USDT-240927.OK  2975430     2.33     1.51  59800.29  149.26  59810.78  149.40  10.4851   7.8345     1.7534  1.3107
BTC-USD-241227.OK   2909331    38.23    31.72  61153.91  159.62  61161.15  159.69   7.2401   4.8610     1.1839  0.7949
BTC-USD-250328.OK   1985694    15.99    14.77  62988.66  179.15  62998.54  179.78   9.8858   5.5868     1.5692  0.8863
BTC-USDT-241227.OK  1810595     2.17     1.54  61323.58  163.41  61339.35  163.33  15.7672   9.2465     2.5713  1.508

## Tips for Large Data Requests

1. **Date Range Limits**: 
   - Daily aggregation: max 20 days per request
   - Monthly aggregation: max 20 months per request

2. **Multiple Instrument Families**: 
   - You can request multiple families: `'BTC-USDT,ETH-USDT'`
   - Use `'ANY'` for all instruments (only with modules 1, 2, 3 and daily aggregation)

3. **Rate Limiting**: 
   - Add delays between requests to avoid rate limits
   - The `fetch_options_data_range` function includes automatic delays

4. **Data Storage**: 
   - The API returns URLs to CSV files that need to be downloaded
   - Consider caching downloaded data locally for repeated analysis

5. **Module Selection for Pricing Models**:
   - **Module 1 (Trade History)**: Best for actual transaction prices and volumes
   - **Module 2 (Candlestick)**: Good for OHLC analysis and quick price snapshots
   - **Module 6 (Orderbook)**: Best for bid-ask spreads and liquidity analysis (single day only)
